# Special Topics in Computer Science
## INT3121 20 UET ----- 4-6 Wednesday 210 GD3
## Week 5

## Diep Ng., Ph.D.
#### Mail-to: ngocdiep at vnu.edu.vn

In [2]:
!python --version

Python 3.6.8 :: Anaconda, Inc.


# 1. ConvNet as feature extractor

VGG16 architecture
<img src="vgg16.png" width=600 alt="VGG16 architecture" />


In [17]:
from keras import preprocessing
from keras.applications.vgg16 import VGG16, preprocess_input
import numpy as np


#load the model & pretrained weight
model = VGG16(weights='imagenet', include_top=True)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [18]:
#preprocessing input image
iname = "flower.jpg"
img = preprocessing.image.load_img(iname, target_size=(224, 224))
x = preprocessing.image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(x.shape)

(1, 224, 224, 3)


In [19]:
#predict
pred = model.predict(x)

In [20]:
pred.shape

(1, 1000)

## Method 1: Remove top layers when loading pretrained model

In [21]:
model = VGG16(weights='imagenet', include_top=False)
pred = model.predict(x)
pred.shape

(1, 7, 7, 512)

In [22]:
features = pred.flatten()
features.shape

(25088,)

## Method 2: Build a model out of the pretrained model

In [23]:
from keras.models import Model

basemodel = VGG16(weights='imagenet')
inputs = basemodel.input
outputs = basemodel.get_layer("fc1").output

model = Model(inputs=inputs, outputs=outputs)

features = model.predict(x)
features.shape

(1, 4096)

In [24]:
#get features from arbitrary intermediate layer
outputs = basemodel.get_layer("block2_pool").output

model = Model(inputs=inputs, outputs=outputs)

features = model.predict(x)
features.shape

(1, 56, 56, 128)

# 2. Fine-tune a model

In [27]:
from keras.layers import * 

#assume, we need to predict 10 classes
num_classes = 10

basemodel = VGG16(weights='imagenet')

#input is the same
inputs = basemodel.input

#use some models only
x = basemodel.get_layer("block5_pool").output

#add some more layer for prediction
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

### Freeze some top layers of model (do not do backpropagation)


In [29]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_11
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool
19 global_average_pooling2d_2
20 dense_3
21 dense_4


In [31]:
#freeze from block4
for layer in model.layers[:15]:
    layer.trainable = False
for layer in model.layers[15:]:
    layer.trainable = True

#now compile and fit as usual
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])
#model.fit(...)

# 3. Different input size 

In [35]:

basemodel = VGG16(input_shape=(256, 300, 3), weights='imagenet', include_top=True) #cause error because of FC layers

model = Model(inputs=basemodel.input, outputs=basemodel.output )


ValueError: When setting `include_top=True` and loading `imagenet` weights, `input_shape` should be (224, 224, 3).

In [37]:
basemodel = VGG16(input_shape=(256, 300, 3), weights='imagenet', include_top=False)
x = basemodel.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=basemodel.input, outputs=outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 256, 300, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 300, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 300, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 150, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 150, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 150, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 75, 128)       0         
__________

## Too small input size?

In [39]:
basemodel = VGG16(input_shape=(20,30,3), weights='imagenet', include_top=False)
x = basemodel.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=basemodel.input, outputs=outputs)

model.summary()

ValueError: Input size must be at least 32x32; got `input_shape=(20, 30, 3)`

In [51]:
basemodel = VGG16(input_shape=(48, 48, 3), weights='imagenet', include_top=False)
x = basemodel.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
outputs = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=basemodel.input, outputs=outputs)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 48, 48, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 48, 48, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 48, 48, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 24, 24, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 24, 24, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 24, 24, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 12, 128)       0         
__________